# SHAP for Linear Models

### Loading Libraries

In [6]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# Data Visualization
import seaborn as sns

# SHAP

# Tabulate
from tabulate import tabulate

### Retrieving The Wine Data

In [2]:
url = 'https://archive.ics.uci.edu/ml/' \
        'machine-learning-databases/' \
        'wine-quality/winequality-white.csv'

In [3]:
file_name = 'wine.csv'

In [4]:
# Check if the file exists in the current directory, as follows:
try:
    wine = pd.read_csv(file_name)
except FileNotFoundError:
    print(f'Downloading {file_name} from {url}...')
    wine = pd.read_csv(url, sep=";")
    wine.to_csv(file_name, index=False)
    print('Download Complete')

Download Complete


In [7]:
summary = wine.describe().transpose().round(2)

In [8]:
summary = summary.drop("count", axis=1)

In [9]:
# Setting Markdown Table Up
markdown_table = tabulate(
    summary, headers='keys', tablefmt='pipe'
)

In [10]:
print(markdown_table)

|                      |   mean |   std |   min |    25% |    50% |    75% |    max |
|:---------------------|-------:|------:|------:|-------:|-------:|-------:|-------:|
| fixed acidity        |   6.85 |  0.84 |  3.8  |   6.3  |   6.8  |   7.3  |  14.2  |
| volatile acidity     |   0.28 |  0.1  |  0.08 |   0.21 |   0.26 |   0.32 |   1.1  |
| citric acid          |   0.33 |  0.12 |  0    |   0.27 |   0.32 |   0.39 |   1.66 |
| residual sugar       |   6.39 |  5.07 |  0.6  |   1.7  |   5.2  |   9.9  |  65.8  |
| chlorides            |   0.05 |  0.02 |  0.01 |   0.04 |   0.04 |   0.05 |   0.35 |
| free sulfur dioxide  |  35.31 | 17.01 |  2    |  23    |  34    |  46    | 289    |
| total sulfur dioxide | 138.36 | 42.5  |  9    | 108    | 134    | 167    | 440    |
| density              |   0.99 |  0    |  0.99 |   0.99 |   0.99 |   1    |   1.04 |
| pH                   |   3.19 |  0.15 |  2.72 |   3.09 |   3.18 |   3.28 |   3.82 |
| sulphates            |   0.49 |  0.11 |  0.22 |   0.

### Fitting a Linear Regression Model